In [8]:
from Data import Depthset, Agument, toTensor
from models import Model

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

import mat73
import kornia
import pickle
import time

In [2]:
file_path = "./data/nyu_depth_v2_labeled.mat"
data_dict = mat73.loadmat(file_path)

dataset = Depthset(
    data_dict=data_dict,
    transform=transforms.Compose([Agument(probability=0.5),toTensor()])
    )
print("dataset loaded")

ERROR:root:ERROR: MATLAB type not supported: containers.Map, (uint32)


dataset loaded


In [10]:
model = Model().cuda()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

epochs = 50
lr = 0.0001
batchsize = 8

train_loader = DataLoader(
    dataset=dataset,
    batch_size=batchsize,
    shuffle=True
    )

optimizer = torch.optim.Adam(model.parameters(),lr)
criterion = nn.L1Loss()
ssim = kornia.losses.SSIMLoss(window_size = 11, max_val = 1000.0 / 10.0, reduction='none')

losses = []
loss_epoch = []

for epoch in range(epochs):
    cum_loss = 0
    counter = 0

    
    model.train()
    start = time.time()
    for i,batch in enumerate(train_loader):
        optimizer.zero_grad()

        image = batch['image'].cuda()
        depth = batch['depth'].cuda()

        depth = 1000.0 / depth

        out = model(image)

        loss = criterion(out, depth)
        l_ssim = torch.clamp((1 - ssim(out, depth)) * 0.5, 0, 1)
        loss = (1.0 * l_ssim.mean().item()) + (0.1 * loss)

        losses.append(loss.item())
        cum_loss += loss.item()
        counter += 1

        loss.backward()
        optimizer.step()

    loss_epoch.append(cum_loss/counter)

    with open("./trained_models/loss_log",'wb') as fp:
        pickle.dump(losses,fp)

    with open("./trained_models/epoch_loss_log",'wb') as fp:
        pickle.dump(loss_epoch,fp)

    path = f"./trained_models/{epoch}.pth"
    torch.save(model.state_dict(),path)

    end = time.time()

    print(f"[{epoch}/{epochs}] loss: {cum_loss} time: {end - start}")
    


[0/50] loss: 534.6158138513565 time: 195.16491174697876


KeyboardInterrupt: 

In [5]:
loss.item()

1.900164008140564

In [ ]:
3 * 50